In [ ]:
%reset -f

In [ ]:
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/digits-mini-dataset-5500/drawings.csv')

In [ ]:
df.head()

In [ ]:
df =  df.sample(frac=1, random_state=0)

In [ ]:
labels = np.float32(df.pop('label'))
pics = np.float32(df.to_numpy())

In [ ]:
pics.shape

In [ ]:
pics = np.reshape(pics,(-1,28,28))

In [ ]:
pics.shape

In [ ]:
import matplotlib.pyplot as plt
pic = pics[0]

plt.figure()
plt.imshow(pic,clim = [0,1],cmap = 'gray' )


In [ ]:
x_train = pics[0 : 4500]
x_test = pics[4500 :]

In [ ]:
import keras
from keras.utils import to_categorical
from tensorflow.keras import layers 

In [ ]:
num_classes = 10

labels = to_categorical(labels,num_classes)
y_train = labels[0 : 4500]
y_test = labels[4500 :]

In [ ]:
model = keras.Sequential([
        keras.Input(shape=(28,28,1)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='model.png',show_shapes = True)

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
callback = EarlyStopping(monitor = 'val_accuracy',restore_best_weights = True, patience = 10)

In [ ]:
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [ ]:
x_train.shape

In [ ]:
x_train = np.expand_dims(x_train, axis=-1)
x_test = np.expand_dims(x_test, axis=-1)

In [ ]:
x_train.shape

In [ ]:
EPOCHS = 20

history = model.fit(x_train,y_train,epochs = EPOCHS, validation_split = 0.1, verbose = True ,callbacks = [callback])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
result = model.evaluate(x_test,y_test)
test_acc = result[1]
test_loss = result[0]

print('test_acc : ',test_acc)
print('test_loss : ',test_loss)

In [ ]:
# calcolo la matrice della confusione 

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

y_test_pred = np.argmax(model.predict(x_test), 1)
matrix = confusion_matrix(np.argmax(y_test,1), y_test_pred) 
print(matrix)

In [ ]:
# visualizzo la matrice della confusione

display = ConfusionMatrixDisplay(matrix)
display.plot(cmap = 'ocean', values_format = '5')